<a href="https://colab.research.google.com/github/lucasgneccoh/SGD_ICP/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Authors implementation

In [ ]:
!git clone https://bitbucket.org/fafz/sgd_icp.git

Cloning into 'sgd_icp'...
remote: Counting objects: 139, done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 139 (delta 95), reused 0 (delta 0)
Receiving objects: 100% (139/139), 160.43 KiB | 2.43 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [ ]:
%%capture
!sudo apt install libeigen3-dev

In [ ]:
%%capture
!sudo apt install libpcl-dev

In [ ]:
%cd /content/sgd_icp
!mkdir build
%cd build
!cmake -DCMAKE_BUILD_TYPE=Release /content/sgd_icp
!make

/content/sgd_icp
mkdir: cannot create directory ‘build’: File exists
/content/sgd_icp/build
-- Boost version: 1.65.1
-- Found the following Boost libraries:
--   program_options
--   system
-- Checking for module 'eigen3'
--   Found eigen3, version 3.3.4
-- Found eigen: /usr/include/eigen3  
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Boost version: 1.65.1
-- Found the following Boost libraries:
--   system
--   filesystem
--   thread
--   date_time
--   iostreams
--   serialization
--   chrono
--   atomic
--   regex
-- Checking for module 'flann'
--   Found flann, version 1.9.1
-- Found Flann: /usr/lib/x86_64-linux-gnu/libflann_cpp_s.a  
-- Checking for module 'libopenni'
--   Found libopen

# Python implementation using code from the course